### 环境准备
由于需要模型，所以下面会安装 ollama 用于 langextract 模型调用。

In [1]:
# Global dependencies required by this notebook:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
from dotenv import load_dotenv

load_dotenv()

True

要使用 Ollama，它需要作为后台服务与脚本并行运行。由于 Jupyter Notebook 设计为顺序执行代码块，这使得同时运行两个代码块变得困难。作为变通方案，我们将使用 Python 的 subprocess 创建服务，确保其不会阻塞任何单元格的执行。

通过命令 `ollama serve` 可启动服务。

`time.sleep(5)` 添加了延迟，确保 Ollama 服务启动完成后再下载模型。

In [26]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

time=2026-01-12T10:31:05.012Z level=INFO source=routes.go:1554 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/codespace/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_REMOTES:[ollama.com] OLLAM

**拉取模型**

使用 `ollama pull qwen3:0.6b` 下载 LLM 模型。

其他模型请访问 https://ollama.com/library

In [23]:
!ollama pull qwen3:0.6b

[GIN] 2026/01/12 - 10:30:03 | 200 |      77.905µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ [GIN] 2026/01/12 - 10:30:04 | 200 |  969.684449ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest 
pulling 7f4030143c1c: 100% ▕██████████████████▏ 522 MB                         
pulling ae370d884f10: 100% ▕██████████████████▏ 1.7 KB                         
pulling d18a5cc71b84: 100% ▕██████████████████▏  11 KB                         
pulling cff3f395ef37: 100% ▕██████████████████▏  120 B                         
pulling b0830f4ff6a0: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest 
writing manifest 
success 


### langextract 示例
安装依赖：

In [4]:
%pip install langextract[openai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 29.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.2 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.8 MB/s  0:00:006m0:00:01
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/30 [websockets]  WARNING: The script websockets is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  5/30 [numpy]uf]  WARNING: The scripts f2py and numpy-config are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 14/30 [yarl]e-crc32c]  WARNING: The scripts pyrsa-decrypt, pyrs

In [27]:
import langextract as lx
import textwrap

# 手动指定模型和提供者
config = lx.factory.ModelConfig(
    model_id="qwen3:0.6b",
    provider="OpenAILanguageModel",
    provider_kwargs={
        "base_url": "http://localhost:11434/v1",
        "api_key": os.getenv("API_KEY"),
    },
)

model = lx.factory.create_model(config)

# 1. 定义提示词和提取规则
prompt = textwrap.dedent("""\
    Extract characters, emotions, and relationships in order of appearance.
    Use exact text for extractions. Do not paraphrase or overlap entities.
    Provide meaningful attributes for each entity to add context.""")

# 2. 提供示例来指导模型
examples = [
    lx.data.ExampleData(
        text="ROMEO. But soft! What light through yonder window breaks? It is the east, and Juliet is the sun.",
        extractions=[
            lx.data.Extraction(
                extraction_class="character",
                extraction_text="ROMEO",
                attributes={"emotional_state": "wonder"},
            ),
            lx.data.Extraction(
                extraction_class="emotion",
                extraction_text="But soft!",
                attributes={"feeling": "gentle awe"},
            ),
            lx.data.Extraction(
                extraction_class="relationship",
                extraction_text="Juliet is the sun",
                attributes={"type": "metaphor"},
            ),
        ],
    )
]

# Run the extraction
lx.extract(
    text_or_documents="Lady Juliet gazed longingly at the stars, her heart aching for Romeo",
    prompt_description=prompt,
    examples=examples,
    model=model,
)

/usr/local/python/3.12.1/lib/python3.12/site-packages/langextract/__init__.py:55: UserWarning: 'use_schema_constraints' is ignored when 'model' is provided. The model should already be configured with schema constraints.
  return extract_func(*args, **kwargs)
LangExtract: model=qwen3:0.6b, current=68 chars, processed=0 chars:  [00:00]

time=2026-01-12T10:31:14.655Z level=WARN source=cpu_linux.go:130 msg="failed to parse CPU allowed micro secs" error="strconv.ParseInt: parsing \"max\": invalid syntax"
time=2026-01-12T10:31:14.734Z level=INFO source=server.go:245 msg="enabling flash attention"
time=2026-01-12T10:31:14.734Z level=INFO source=server.go:429 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model /home/codespace/.ollama/models/blobs/sha256-7f4030143c1c477224c5434f8272c662a8b042079a0a584f0a27a1684fe2e1fa --port 35701"
time=2026-01-12T10:31:14.735Z level=INFO source=sched.go:443 msg="system memory" total="15.6 GiB" free="10.1 GiB" free_swap="0 B"
time=2026-01-12T10:31:14.735Z level=INFO source=server.go:746 msg="loading model" "model layers"=29 requested=-1
time=2026-01-12T10:31:14.748Z level=INFO source=runner.go:1405 msg="starting ollama engine"
time=2026-01-12T10:31:14.748Z level=INFO source=runner.go:1440 msg="Server listening on 127.0.0.1:35701"
time=2026-01-12T10:31:14.757Z leve

[GIN] 2026/01/12 - 10:32:00 | 200 |  46.40237364s |       127.0.0.1 | POST     "/v1/chat/completions"


AnnotatedDocument(extractions=[Extraction(extraction_class='character', extraction_text='Lady Juliet', char_interval=CharInterval(start_pos=0, end_pos=11), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=1, group_index=0, description=None, attributes={'emotional_state': 'longingly'}), Extraction(extraction_class='emotion', extraction_text='gazed longingly', char_interval=CharInterval(start_pos=12, end_pos=27), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=2, group_index=1, description=None, attributes={'feeling': 'longing'}), Extraction(extraction_class='relationship', extraction_text='her heart aching for Romeo', char_interval=CharInterval(start_pos=42, end_pos=68), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=3, group_index=2, description=None, attributes={'type': 'metaphor'})], text='Lady Juliet gazed longingly at the stars, her heart aching for Romeo')